In [ ]:
# BLockchain DB Development

In [ ]:
! whoami

In [ ]:
!pip install plyvel configparser ecdsa

In [2]:
import os, sys
import mmap
import struct
import pickle
import stat
import plyvel
from configparser import ConfigParser
from blockchain_index import DBBlockIndex

from btc_utils import BTCUtils as Utils
from btcpy.btcpy.structs import block as btcpy_block
from btcpy.btcpy.setup import setup

from db_structs.db_blockchain import DBBlockchain, DBBlock, DBTx, DBTxo, DBTxi, DBDirectory

import sys
import importlib  
# sys.path.append('../python-bitcoin-blockchain-parser')
# from blockchain_parser import block #, blockchain


setup('mainnet')
BITCOIN_CONSTANT = b"\xf9\xbe\xb4\xd9"


class BlockchainDB:
    _CURRENT_BLOCK_IDX = ''
    _CURRENT_BLOCK = ''
    _BLOCK_INDEXES = ''
    _CONFIG_FILE = 'app.conf'
    _CONFIG = []
    blockIndexes = ''    
    BLKCHN_FILE=''
    path_index = ''
    
    def __init__(self):
        self.load_config();
        
        self.PATH_BTC_ROOT = self._CONFIG_BLOCKCHAIN['path_btc_root']
        self._PATH_BLKS_FOLDER = self._CONFIG_BLOCKCHAIN['path_btc_blocks']
        self._PATH_INDEX_FOLDER = self._CONFIG_BLOCKCHAIN['path_btc_index']
        self._PATH_CHAINSTATE_FOLER = self._CONFIG_BLOCKCHAIN['path_btc_chainstate']
        self._PATH_TXINDEX_FOLDER = self._CONFIG_BLOCKCHAIN['path_btc_txindex']  
        pass;
    
    # Constant separating blocks in the .blk files
    
    def load_config(self):
        self._CONFIG_MDB = Utils.load_config(self._CONFIG_FILE, 'mdb')
        self._CONFIG_BLOCKCHAIN = Utils.load_config(self._CONFIG_FILE, 'blockchain')
        self._CONFIG_RAY = Utils.load_config(self._CONFIG_FILE, 'ray')

    def get_files(self, path):
        """
        Given the path to the .bitcoin directory, returns the sorted list of .blk
        files contained in that directory
        """
        if not stat.S_ISDIR(os.stat(path)[stat.ST_MODE]):
            return [path]
        files = os.listdir(path)
        files = [f for f in files if f.startswith("blk") and f.endswith(".dat")]
        files = map(lambda x: os.path.join(path, x), files)
        return sorted(files)


    def get_blocks(self, blockfile):
        """
        Given the name of a .blk file, for every block contained in the file,
        yields its raw hexadecimal value
        """
        with open(blockfile, "rb") as f:
            if os.name == 'nt':
                size = os.path.getsize(f.name)
                raw_data = mmap.mmap(f.fileno(), size, access=mmap.ACCESS_READ)
            else:
                # Unix-only call, will not work on Windows, see python doc.
                raw_data = mmap.mmap(f.fileno(), 0, prot=mmap.PROT_READ)
            length = len(raw_data)
            offset = 0
            block_count = 0
            while offset < (length - 4):
                if raw_data[offset:offset+4] == BITCOIN_CONSTANT:
                    offset += 4
                    size = struct.unpack("<I", raw_data[offset:offset+4])[0]
                    offset += 4 + size
                    block_count += 1
                    yield raw_data[offset-size:offset]
                else:
                    offset += 1
            raw_data.close()


    def get_block(self, blockfile, offset):
        """Extracts a single block from the blockfile at the given offset"""
        with open(blockfile, "rb") as f:
            f.seek(offset - 4)  # Size is present 4 bytes before the db offset
            size, = struct.unpack("<I", f.read(4))
            return f.read(size)
    
    def load_blkchn_file(self, blkchn_index=1):
        self.BLKCHN_FILE = '{0}/blk{1}.dat'.format(self._PATH_BLKS_FOLDER,'{:05d}'.format(blkchn_index));
        
    def getBlockIndexes(self, index):
        """There is no method of leveldb to close the db (and release the lock).
        This creates problem during concurrent operations.
        This function also provides caching of indexes.
        """
        if self.path_index != index:
            db = plyvel.DB(index, compression=None, create_if_missing=True)
            self.blockIndexes = [DBBlockIndex(Utils.format_hash(k[1:]), v)
                                 for k, v in db.iterator() if k[0] == ord('b')]
            db.close()
            self.blockIndexes.sort(key=lambda x: x.height)
            self.path_index = index
        return self.blockIndexes
    
    def get_ordered_blocks(self, index, start=0, end=None, cache=None):
        """Yields the blocks contained in the .blk files as per
        the heigt extract from the leveldb index present at path
        index maintained by bitcoind.
        """

        blockIndexes = None

        if cache and os.path.exists(cache):
            # load the block index cache from a previous index
            with open(cache, 'rb') as f:
                blockIndexes = pickle.load(f)
            print("Loading existing cache file {0} \n".format(cache))

        if blockIndexes is None:
            # build the block index
            blockIndexes = self.getBlockIndexes(index)
            if cache and not os.path.exists(cache):
                # cache the block index for re-use next time
                print("Creating new cache file {0} \n".format(cache))
                with open(cache, 'wb') as f:
                    pickle.dump(blockIndexes, f)

        # remove small forks that may have occured while the node was live.
        # Occassionally a node will receive two different solutions to a block
        # at the same time. The Leveldb index saves both, not pruning the
        # block that leads to a shorter chain once the fork is settled without
        # "-reindex"ing the bitcoind block data. This leads to at least two
        # blocks with the same height in the database.
        # We throw out blocks that don't have at least 6 other blocks on top of
        # it (6 confirmations).
        orphans = []  # hold blocks that are orphans with < 6 blocks on top
        last_height = -1
        for i, blockIdx in enumerate(blockIndexes):
            if last_height > -1:
                # if this block is the same height as the last block an orphan
                # occurred, now we have to figure out which of the two to keep
                if blockIdx.height == last_height:

                    # loop through future blocks until we find a chain 6 blocks
                    # long that includes this block. If we can't find one
                    # remove this block as it is invalid
                    if self._index_confirmed(blockIndexes[i:]):

                        # if this block is confirmed, the unconfirmed block is
                        # the previous one. Remove it.
                        orphans.append(blockIndexes[i - 1].hash)
                    else:

                        # if this block isn't confirmed, remove it.
                        orphans.append(blockIndexes[i].hash)

            last_height = blockIdx.height

        # filter out the orphan blocks, so we are left only with block indexes
        # that have been confirmed
        # (or are new enough that they haven't yet been confirmed)
        blockIndexes = list(filter(lambda block: block.hash not in orphans, blockIndexes))

        if end is None:
            end = len(blockIndexes)

        if end < start:
            blockIndexes = list(reversed(blockIndexes))
            start = len(blockIndexes) - start
            end = len(blockIndexes) - end
        
        _PREV_BLK_FILE = -1
        for blkIdx in blockIndexes[start:end]:
            self._CURRENT_BLOCK_IDX = blkIdx;
            if blkIdx.file == -1 or blkIdx.data_pos == -1:
                break
            #print("BlkIndex: ", json.dumps(blkIdx.to_json(), indent=2))
            blkFile = os.path.join(self._PATH_BLKS_FOLDER, "blk%05d.dat" % blkIdx.file)

            # block struct from python_bitcoin_blockchain_parser
            #yield block.Block(self.get_block(blkFile, blkIdx.data_pos), blkIdx.height)
            #######################################
            if self._CURRENT_BLOCK_IDX.file > _PREV_BLK_FILE :
                db_blkchn = DBBlockchain();
                db_blkchn.from_btcpy_obj(self._CURRENT_BLOCK_IDX);
                _PREV_BLK_FILE = self._CURRENT_BLOCK_IDX.file;
                del db_blkchn;
            ######################################
            
            #block_struct from btcpy block
            yield btcpy_block.Block.deserialize(self.get_block(blkFile, blkIdx.data_pos))
            
            
            
    def parse(self):
        #blkchn = blockchain.Blockchain(os.path.expanduser(self._BLK_FOLDER_PATH))
        for blk in self.get_ordered_blocks(os.path.expanduser(self._PATH_INDEX_FOLDER), end = 100):
            print(blk.header)
    
    @staticmethod       
    def remove_blkchain_lock():
        LOCK_FILE ="/root/.bitcoin/blocks/index/LOCK"

        ## If file exists, delete it ##
        if os.path.isfile(LOCK_FILE):
            os.remove(LOCK_FILE)
        else:    ## Show an error ##
            print("Error: %s file not found" % LOCK_FILE)
        

In [3]:
BlockchainDB.remove_blkchain_lock()
#!sudo rm -rf /root/.bitcoin/blocks/index/LOCK

In [ ]:
import json
from db_structs.db_blockchain import global_connect, global_disconnect, DBBlockchain, DBBlock, DBTx, DBTxo, DBTxi, DBUtxo, DBDirectory

global_connect()

blkchain = BlockchainDB();
for blk in blkchain.get_ordered_blocks(os.path.expanduser(blkchain._PATH_INDEX_FOLDER), start=298851, end=666000):
    blkchain._CURRENT_BLOCK = blk;
    #print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
    #print(json.dumps(blk.header.to_json(), indent=2))
    ###################################
    db_blk=DBBlock()
    db_blk.from_btcpy_obj(blkchain._CURRENT_BLOCK_IDX)
    del db_blk
    #################################
    
   
    #print("##########################################################################################")
    for txn_obj in blk.txns:
        #print(json.dumps(txn_obj.to_json(), indent=2))
        ################################################
        db_utxo = DBUtxo();
        db_utxo.from_btcpy_obj(blkchain._CURRENT_BLOCK_IDX, txn_obj)
        del db_utxo
        ###############################################
        
           
global_disconnect()    
print("All blocks processed")
#! ./remove_btc_index_lock.sh

In [1]:
pwd

'/workdir/Tutorials/Crypto/Bitcoin/apps/blockchain_db'

In [ ]:
#create cache file

import json
from db_structs.db_blockchain import global_connect, global_disconnect, DBBlockchain, DBBlock, DBTx, DBTxo, DBTxi, DBUtxo, DBDirectory

global_connect()

blkchain = BlockchainDB();
for blk in blkchain.get_ordered_blocks(os.path.expanduser(blkchain._PATH_INDEX_FOLDER), start=0, end=600000, cache='/workdir/Tutorials/Crypto/Bitcoin/apps/blockchain_db/cache/idx_cache'):
    blkchain._CURRENT_BLOCK = blk;
    #print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
    #print(json.dumps(blk.header.to_json(), indent=2))
    ###################################
    db_blk=DBBlock()
    db_blk.from_btcpy_obj(blkchain._CURRENT_BLOCK_IDX)
    del db_blk
    #################################
    
   
    #print("##########################################################################################")
    #for txn_obj in blk.txns:
        #print(json.dumps(txn_obj.to_json(), indent=2))
        ################################################
        #db_utxo = DBUtxo();
        #db_utxo.from_btcpy_obj(blkchain._CURRENT_BLOCK_IDX, txn_obj)
        #del db_utxo
        ###############################################
        
           
global_disconnect()    
print("All blocks processed")
#! ./remove_btc_index_lock.sh

In [ ]:

def parse_block(blk_obj, blkchain_idx, q_del):
    db_blk=DBBlock();
    db_blk.form_btcpy_obj(blkchain_idx);
    for txn_obj in blk_obj.txns:
        db_utxo = DBUtxo();
        db_utxo.from_btcpy_obj(blkchain_idx, txn_obj);
    del db_blk;
    del db_utxo;
    
    

In [ ]:
%%writefile 'btc_utils.py'

from binascii import hexlify
import hashlib, codecs
from configparser import ConfigParser
from bson.binary import Binary as BsonBinary

class BTCUtils(object):
    
    @staticmethod
    def load_config(config_file, section_name):
        config = ConfigParser()
        config.read(config_file)
        return config[section_name]
    
    @staticmethod
    def b2x(b):
        return b.hex();
    
    @staticmethod
    def s2x(s):
        return hexlify(bytes(s, 'utf-8'))
    
    @staticmethod
    def x2b(x):
        return codecs.decode(x, 'hex')
    
    @staticmethod
    def x2bson(x):
        return BsonBinary(BTCUtils.x2b(x), 5)
    
    @staticmethod
    def s2bson(s):
        return BTCUtils.x2bson(BTCUtils.s2x(s))
    
    @staticmethod
    def i2x(i,nBits):
        return hex(i)[2:].zfill(nBits);
    
    @staticmethod
    def seed2pvk(s_str, nHash=1):
        seed_b = s.encode('utf8') #converts string to bytes
        for x in range(nHash):
            sha_pvk = hashlib.sha256(seed_b)
            seed_b = sha_pvk;
        pvk_b = sha_pvk.digest()
        return pvk_b
    
    @staticmethod
    def btc_ripemd160(data):
        h1 = hashlib.sha256(data).digest()
        r160 = hashlib.new("ripemd160")
        r160.update(h1)
        return r160.digest()

    @staticmethod
    def double_sha256(data):
        return hashlib.sha256(hashlib.sha256(data).digest()).digest()

    @staticmethod
    def format_hash(hash_):
        return str(hexlify(hash_[::-1]).decode("utf-8"))

    @staticmethod
    def decode_uint32(data):
        assert(len(data) == 4)
        return struct.unpack("<I", data)[0]

    @staticmethod
    def decode_uint64(data):
        assert(len(data) == 8)
        return struct.unpack("<Q", data)[0]

    @staticmethod
    def decode_varint(data):
        assert(len(data) > 0)
        size = int(data[0])
        assert(size <= 255)

        if size < 253:
            return size, 1

        if size == 253:
            format_ = '<H'
        elif size == 254:
            format_ = '<I'
        elif size == 255:
            format_ = '<Q'
        else:
            # Should never be reached
            assert 0, "unknown format_ for size : %s" % size

        size = struct.calcsize(format_)
        return struct.unpack(format_, data[1:size+1])[0], size + 1
    
    @staticmethod
    def format_json(json_obj, indent=2):
        json_formatted_str = json.dumps(json.load(json_obj), indent=indent)
        return json_formatted_str
    
    @staticmethod
    def val_non_satoshi(val_satoshi):
        return val_satoshi/100000000
    
    @staticmethod
    def val_satoshi(val_non_satoshi):
        return val_non_satoshi * 100000000



In [ ]:
%%writefile 'blockchain_index.py'


from struct import unpack

from utils import Utils

BLOCK_HAVE_DATA = 8
BLOCK_HAVE_UNDO = 16


def _read_varint(raw_hex):
    """
    Reads the weird format of VarInt present in src/serialize.h of bitcoin core
    and being used for storing data in the leveldb.
    This is not the VARINT format described for general bitcoin serialization
    use.
    """
    n = 0
    pos = 0
    while True:
        data = raw_hex[pos]
        pos += 1
        n = (n << 7) | (data & 0x7f)
        if data & 0x80 == 0:
            return n, pos
        n += 1


class DBBlockIndex(object):
    def __init__(self, blk_hash, raw_hex):
        self.hash = blk_hash
        pos = 0
        n_version, i = _read_varint(raw_hex[pos:])
        pos += i
        self.height, i = _read_varint(raw_hex[pos:])
        pos += i
        self.status, i = _read_varint(raw_hex[pos:])
        pos += i
        self.n_tx, i = _read_varint(raw_hex[pos:])
        pos += i
        if self.status & (BLOCK_HAVE_DATA | BLOCK_HAVE_UNDO):
            self.file, i = _read_varint(raw_hex[pos:])
            pos += i
        else:
            self.file = -1

        if self.status & BLOCK_HAVE_DATA:
            self.data_pos, i = _read_varint(raw_hex[pos:])
            pos += i
        else:
            self.data_pos = -1
        if self.status & BLOCK_HAVE_UNDO:
            self.undo_pos, i = _read_varint(raw_hex[pos:])
            pos += i

        assert(pos + 80 == len(raw_hex))
        self.version, p, m, time, bits, self.nonce = unpack(
            "<I32s32sIII",
            raw_hex[-80:]
        )
        self.prev_hash = Utils.format_hash(p)
        self.merkle_root = Utils.format_hash(m)

    def __repr__(self):
        return "DBBlockIndex(%s, height=%d, file_no=%d, file_pos=%d)" \
               % (self.hash, self.height, self.file, self.data_pos)
    
    def to_json(self):
        return {
            'blk_hash': self.hash,
            'prev_hash': self.prev_hash,
            'height': self.height,
            'file': self.file,
            'nTxns': self.n_tx,
            'status': self.status,
            'data_posn': self.data_pos,
            'merkle_root' : self.merkle_root,
            'version': self.version,
            'nounce':self.nonce
        }


In [ ]:
%%writefile 'db_mongo.conf'

[mdb]
    #host = 127.0.0.1
    #host = 192.168.0.229 
    #db_btc = btc_miner
    
    host = 192.168.0.111    
    db_btc = btc_nas    
    
    port = 27017
    username = root
    passwd = chinnus_pwd
    #cln_keybank = btc_keybank
    cln_keybank = btc_keybank_temp
    cln_blocks = btc_blocks
    
[blockchain]
    cln_blockchains = btc_blkchains,
    cln_blocks = btc_blks,
    cln_txns = btc_txs,
    cln_txi = btc_txi,
    cln_txo = btc_txo,
    cln_directory = btc_directory,
    cln_keybank = btc_keybank

[ray]
    mode = worker
    num_cpus = 12
    redis_address = 192.168.0.229
    redis_port = 6379
    redis_passwd = letsdoit

In [ ]:
%%writefile 'db_mongo.py'

import pymongo
from configparser import ConfigParser
from utils import Utils

class DbMongo(object):
    usrName, usrPwd, dbURI, dbPort = 'root', 'pwd', 127.0.0.1 , 27017;
    config = null;
    
    
    mdb_config = Utils.load_config('db_mongo.conf', 'mdb')
    ray_config = Utils.load_config('db_mongo.conf', 'ray')
    blkchain_config = Utils.load_config('db_mongo.conf', 'blockchain')
    mdb_client = DbMongo.db_connect(mdb_config['username'], mdb_config['passwd'], mdb_config['host'], mdb_config['port'])
    mdb_db = DbMongo.db_select(mdb_client, mdb_config['db_btc'])
    mdb_cln_keybank = DbMongo.db_select_cln(mdb_db, blkchain_config['cln_keybank'])
    
    def __init__(self):
        self.config =configparser.ConfigParser();
        
    @staticmethod
    def db_connect(usrName, usrPwd, dbURI, dbPort):
        return pymongo.MongoClient("mongodb:///{0}:{1}@{2}:{3}".format(usrName, usrPwd, dbURI, dbPort))
    
    @staticmethod
    def db_select(dbClient, dbName):
        return dbClient[dbName]
    
    @staticmethod
    def db_select_cln(dbName, collectionName):
        return dbName[collectionName]
    
    @staticmethod
    def db_upload_keypairs(keysCln, pbk_b, pvk_b):
        keysCln.insert_one({'pbk': BsonBinary(pbk_b, 5), 'pvk': BsonBinary(pvk_b, 5)})
        
    @staticmethod    
    def db_upload_ripemd160(keysCln, PbK, PvK, RipeMD160):
        keysCln.insert_one({'pbk': BsonBinary(PbK,5), 'pvk': BsonBinary(PvK,5), 'ripemd160': BsonBinary(RipeMD160,5) })
        
    @staticmethod    
    def db_upload_public_address(keysCln, PbK, PvK, RipeMD160, PbA):
        keysCln.insert_one({'pbk': BsonBinary(PbK,5), 'pvk': BsonBinary(PvK,5), 'ripemd160': BsonBinary(RipeMD160,5), 'pba_b58': BsonBinary(PbA, 5) })
        
    @staticmethod   
    def db_bulk_upload(keysCln, bulkKeypairs):
        keysCln.insert_many(bulkKeypairs)

In [ ]:
%%writefile 'btc_utils.py'

from binascii import hexlify
import hashlib, codecs, struct
from configparser import ConfigParser
from bson.binary import Binary as BsonBinary

class BTCUtils(object):    
    @staticmethod
    def load_config(config_file, section_name):
        config = ConfigParser()
        config.read(config_file)
        return config[section_name]
    
    @staticmethod
    def b2x(b):
        return b.hex();
    
    @staticmethod
    def x2b(x):
        return codecs.decode(x, 'hex')
    
    @staticmethod
    def x2bson(x):
        return BsonBinary(BTCUtils.x2b(x),5) 
    
    @staticmethod
    def i2x(i,nBits):
        return hex(i)[2:].zfill(nBits);
    
    @staticmethod
    def seed2pvk(s_str, nHash=1):
        seed_b = s.encode('utf8') #converts string to bytes
        for x in range(nHash):
            sha_pvk = hashlib.sha256(seed_b)
            seed_b = sha_pvk;
        pvk_b = sha_pvk.digest()
        return pvk_b
    
    @staticmethod
    def btc_ripemd160(data):
        h1 = hashlib.sha256(data).digest()
        r160 = hashlib.new("ripemd160")
        r160.update(h1)
        return r160.digest()

    @staticmethod
    def double_sha256(data):
        return hashlib.sha256(hashlib.sha256(data).digest()).digest()

    @staticmethod
    def format_hash(hash_):
        return str(hexlify(hash_[::-1]).decode("utf-8"))

    @staticmethod
    def decode_uint32(data):
        assert(len(data) == 4)
        return struct.unpack("<I", data)[0]

    @staticmethod
    def decode_uint64(data):
        assert(len(data) == 8)
        return struct.unpack("<Q", data)[0]

    @staticmethod
    def decode_varint(data):
        assert(len(data) > 0)
        size = int(data[0])
        assert(size <= 255)

        if size < 253:
            return size, 1

        if size == 253:
            format_ = '<H'
        elif size == 254:
            format_ = '<I'
        elif size == 255:
            format_ = '<Q'
        else:
            # Should never be reached
            assert 0, "unknown format_ for size : %s" % size

        size = struct.calcsize(format_)
        return struct.unpack(format_, data[1:size+1])[0], size + 1
    
    @staticmethod
    def format_json(json_obj, indent=2):
        json_formatted_str = json.dumps(json_obj, indent=indent)
        return json_formatted_str


In [1]:
%%writefile 'db_structs/db_blockchain.py'

import pymongo as pmng
import mongoengine as mge
from datetime import datetime
import json
from configparser import ConfigParser
from btc_utils import BTCUtils as Utils
from bson.binary import Binary as BsonBinary


db_config = Utils.load_config('app.conf','mdb');
config_data = dict(
    username =  db_config['username'],
    password =  db_config['passwd'],
    host = db_config['host'],
    port =  int(db_config['port']),
    authentication_source=  db_config['auth_db']
)

def global_connect(conn_alias='chinnus_nas'):
    mge.register_connection(alias=conn_alias, name=db_config['db_blockchain'], **config_data)
    
def global_disconnect(conn_alias='chinnus_nas'):
    mge.disconnect(alias=conn_alias)

class DBBlockchain(mge.Document):
    blkchain_idx = mge.IntField();
    blks_cnt  = mge.IntField();
    blk_first  = mge.IntField();
    blk_last  = mge.IntField();
    is_processed  = mge.BooleanField(default=False);
    
    meta = {
        'db_alias' : 'chinnus_nas',
        'collection' : 'btc_blkchains'
    }
    
    def from_btcpy_obj(self, blkIdx_obj):
        self.blkchain_idx = blkIdx_obj.file;
        #DBBlockchain.blks_cnt = BlocksCnt
        #DBBlockchain.blks_first = btcpy_obj;
        #DBBlockchain.blk_last = BlkLast
        self.save()
    
    
    @staticmethod
    def setup(BlockchainIdx, BlocksCnt, BlkFirst, BlkLast):
        DBBlockchain.blkchain_idx = BlockchainIdx;
        DBBlockchain.blks_cnt = BlocksCnt
        DBBlockchain.blks_first = BlkFirst;
        DBBlockchain.blk_last = BlkLast
    
    def to_json(self):
        return json.dumps({
            blkchain_idx : self.blkchain_idx,
            blks_cnt : self.blks_cnt,
            blk_first : self.blk_first,
            blk_last : self.blk_last,
            is_processed : self.is_processed
        }, indent=2)
    
    

class DBBlock(mge.Document):
    blk_height = mge.IntField()
    blk_hash = mge.StringField()
    blk_hash_prev = mge.StringField()
    blkchain_idx = mge.IntField()
    tx_cnt = mge.IntField();
    is_processed = mge.BooleanField()
    
    meta = {
        'db_alias' : 'chinnus_nas',
        'collection' : 'btc_blks'
    }
    
    @staticmethod
    def setup(BlockHeight, BlockHash, PrevBlockHash, BlockchainIdx, TxCnt=0):
        DBBlock.blk_height = BlockHeight,
        DBBlock.blk_hash = BlockHash,
        DBBlock.blk_hash_prev = PrevBlockHash,
        DBBlock.blkchain_idx = BlockchainIdx,
        DBBlock.tx_cnt = TxCnt
    
    def from_btcpy_obj(self, blkIdx_obj):
        self.blk_height = blkIdx_obj.height;
        self.blk_hash = blkIdx_obj.hash;
        self.blk_hash_prev = blkIdx_obj.prev_hash;
        self.blkchain_idx = blkIdx_obj.file;
        self.tx_cnt = blkIdx_obj.n_tx;
        self.save()

    def to_json(self):
        return json.dumps({
            blk_height : self.blk_height,
            blk_hash : self.blk_hash,
            blk_hash_prev : self.blk_hash_prev,
            blkchain_idx : self.blkchain_idx,
            tx_cnt : self.tx_cnt,
            is_processed : self.is_processed
        }, indent=2)

class DBTxi(mge.EmbeddedDocument):
    txi_idx = mge.IntField()
    txi_is_coinbase = mge.BooleanField()
    prev_txo_txid = mge.StringField()
    prev_txo_idx = mge.IntField()
    btc_addr_from = mge.StringField()
    i_script = mge.StringField()
    i_script_asm = mge.StringField()
    i_script_type = mge.StringField(default='scriptsig')
    is_witness_signed = mge.BooleanField()
    
        
    def from_btcpy_obj(self, idx, txi_obj, is_coinbase=False):        
        if is_coinbase:
            self.txi_idx = 0;
            self.txi_is_coinbase = True;
        else:
            self.txi_idx = idx;
            self.txi_is_coinbase = False;        
            self.prev_txo_txid = txi_obj.txid
            self.prev_txo_idx = txi_obj.txout
            #btc_addr_from = mge.StringField()
            self.i_script = txi_obj.script_sig.hexlify()
            self.i_script_asm = str(txi_obj.script_sig)
            self.i_script_type = txi_obj.script_sig.type
            if txi_obj.witness is not None:
                self.is_witness_signed = True;
            
    
        
    def to_json(self):
        return json.dumps({
            txi_idx : self.txi_idx,
            txi_is_coinbase : self.txi_is_coinbse,
            prev_txo_txid : self.prev_txo_txid,
            prev_txo_idx : self.prev_txo_idx,
            btc_addr_from : self.btc_addr_from,
            i_script : self.i_script,
            i_script_asm : self.i_script_asm,
            i_script_type : self.i_script_type
        }, indent=2)
    
    
class DBTxo(mge.EmbeddedDocument):
    txo_idx = mge.IntField();
    btc_addr_to = mge.StringField();
    btc_value = mge.FloatField();
    o_script = mge.StringField();
    o_script_asm = mge.StringField();
    o_script_type = mge.StringField();
    utxo = mge.BooleanField(default=True);
    utxo_txid = mge.StringField()
    
    
    def from_btcpy_obj(self, txo_obj):
        self.txo_idx = txo_obj.n;        
        self.btc_value = Utils.val_non_satoshi(txo_obj.value);
        self.o_script = txo_obj.script_pubkey.hexlify();
        self.o_script_asm = str(txo_obj.script_pubkey);
        self.o_script_type = txo_obj.script_pubkey.type;
        if txo_obj.script_pubkey.address is not None:
            self.btc_addr_to = str(txo_obj.script_pubkey.address());
        self.utxo = True;
        

    
class DBTx(mge.Document):
    txid = mge.StringField();
    blk_height = mge.IntField();
    #btc_addr_from = mge.StringField();
    txi_cnt = mge.IntField();
    txo_cnt = mge.IntField();
    utxo = mge.BooleanField(default=True);
    txis = mge.EmbeddedDocumentListField(DBTxi);
    txos = mge.EmbeddedDocumentListField(DBTxo);
    
    meta = {
        'db_alias' : 'chinnus_nas',
        'collection' : 'btc_txs'
    }
    
    def set_utxo_spent(self, txo_idx, utxo_status=False, utxo_txid=None):
        self.txos[txo_idx].utxo = utxo_status;
        if utxo_txid is not None:
            self.txos[txo_idx].utxo_txid = utxo_txid;
        
    
    def from_btcpy_obj(self, blkidx_obj, txn_obj):
        self.txid = txn_obj.txid;
        self.blk_height = blkidx_obj.height;
        self.txi_cnt = len(txn_obj.ins);
        self.txo_cnt = len(txn_obj.outs);
        self.utxo = True;
        self.txis = [];
        if txn_obj.is_coinbase():
            for txin in txn_obj.ins:
                txi_obj = DBTxi();
                txi_obj.from_btcpy_obj(txn_obj.ins.index(txin), txin, True)
                self.txis.append(txi_obj)
        else:
            for txin in txn_obj.ins:
                txi_obj = DBTxi();
                txi_obj.from_btcpy_obj(txn_obj.ins.index(txin), txin, False)
                self.txis.append(txi_obj)
                update_fields = { f'set__txos__{txin.txout}__utxo': False,
                                 f'set__txos__{txin.txout}__utxo_txid': self.txid}
                
                DBTx.objects.get(txid=txin.txid).update(**update_fields)
        self.txos = [];
        for txout in txn_obj.outs:
            txo_obj = DBTxo();            
            txo_obj.from_btcpy_obj(txout)
            self.txos.append(txo_obj)
        self.save()
        


class DBUtxo(mge.Document):
    txid = mge.StringField();
    txo_idx = mge.IntField();
    txid_oidx = mge.StringField();
    blkchain_idx = mge.IntField();    
    blk_height = mge.IntField();
    btc_addr_to = mge.StringField();
    btc_value = mge.FloatField();
    o_script = mge.StringField();
    o_script_asm = mge.StringField();
    o_script_type = mge.StringField();
    is_utxo = mge.BooleanField();
    
    
    def delete_spent(self, i_txid, io_idx):
        #DBUtxo.objects.get(txid=i_txid, txo_idx=io_idx).delete()
        txid_oidx_key = i_txid + "-" + str(io_idx);    
            
        try:
            obj_to_delete = DBUtxo.objects(txid_oidx = txid_oidx_key ).first();
            if obj_to_delete is not None:
                obj_to_delete.delete()
        except: #(mge.DoesNotExist) as e:
            #print("Does not exist error{0}: {1}".format(e.args, txid_oidx_key,))
            print("Does not exist error: {0}".format(txid_oidx_key))
        
        
    def del_from_btcpy_obj(self, txn_obj):
        if not txn_obj.is_coinbase():
            for txin in txn_obj.ins:
                self.delete_spent(txin.txid, txin.txout)
    
       
    def add_from_btcpy_obj(self, blkidx_obj, txn_obj, txo_obj):
        try:
            utxo_obj = DBUtxo();
            utxo_obj.txid = txn_obj.txid;
            utxo_obj.txo_idx = txo_obj.n;
            utxo_obj.txid_oidx = txn_obj.txid + "-" + str(txo_obj.n);
            utxo_obj.blkchain_idx = blkidx_obj.file;
            utxo_obj.blk_height = blkidx_obj.height;
            if txo_obj.script_pubkey.address() is not None:
                utxo_obj.btc_addr_to = str(txo_obj.script_pubkey.address());
            utxo_obj.btc_value = Utils.val_non_satoshi(txo_obj.value);
            utxo_obj.o_script = txo_obj.script_pubkey.hexlify();
            utxo_obj.o_script_asm = str(txo_obj.script_pubkey);
            utxo_obj.o_script_type = txo_obj.script_pubkey.type;
            utxo_obj.is_utxo = True;
            utxo_obj.save();
        except (pmng.errors.DuplicateKeyError, mge.errors.NotUniqueError) as e:
            print("Duplicate key error({0}): {1}".format(e.args, utxo_obj.txid_oidx))
        
    def from_btcpy_obj(self, blkidx_obj, txn_obj):      
        for txo_obj in txn_obj.outs:
            self.add_from_btcpy_obj(blkidx_obj, txn_obj, txo_obj);
            
        if not txn_obj.is_coinbase():
            for txi_obj in txn_obj.ins:
                self.delete_spent(txi_obj.txid, txi_obj.txout);
                
    def from_btcpy_obj_mp(self, blkidx_obj, txn_obj, q_del):
        for txo_obj in txn_obj.outs:
            self.add_from_btcpy_obj(blkidx_obj, txn_obj, txo_obj);
            
        if not txn_obj.is_coinbase():
            for txi_obj in txn_obj.ins:                
                q_del.put(txi_obj.txid + "-" + str(txi_obj.txout)); #for non-coinbse txns, add txid_oidx to q for future handling

                
    meta = {
        'db_alias' : 'chinnus_nas',
        'collection' : 'btc_utxos'
    }
    
    

class DBDirectory(mge.Document):
    btc_addr = mge.StringField();
    btc_addr_type = mge.StringField();
    btc_addr_is_segwit = mge.BooleanField();
    btc_addr_p2pkh = mge.StringField();
    btc_addr_p2wpkh = mge.StringField();
    btc_addr_p2sh = mge.StringField();
    btc_addr_p2wsh = mge.StringField();
    btc_addr_bech32 = mge.StringField();
    pbk = mge.BinaryField();
    hash160 = mge.BinaryField();
    pvk = mge.BinaryField();
    bal = mge.FloatField();
    bal_asof = mge.DateTimeField();
    utxo = mge.BooleanField(default=True);
    txis = mge.ListField()
    txos = mge.ListField()
    
    meta = {
        'db_alias' : 'chinnus_nas',
        'collection' : 'btc_directory'
    }


Overwriting db_structs/db_blockchain.py


In [ ]:
import binascii
st = "somdd"
print("Bytes: ", bytes(st, 'utf-8'))

print("hex ", binascii.hexlify(bytes(st, 'utf-8')))

In [ ]:
from btc_utils import BTCUtils as Utils

hash_s = "000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f"
bson_bin = Utils.s2bson(hash_s)
print(type(bson_bin))

In [ ]:
Utils.x2b(Utils.s2x(hash_s))

In [ ]:
vins = [
    {
      "coinbase": {
        "hex": "048bdb051a025a03062f503253482f"
      },
      "sequence": "4294967295"
    },{'script':"scrisps"}
  ]

In [ ]:
from db_structs.db_blockchain import DBTxi, DBTxo, DBTx

[DBTxi().from_btcpy_obj(vins.index(vin), vin, True) for vin in vins]

In [ ]:
idx = 2;
print('set__txos__{0}__utxo'.format(idx))

In [ ]:
class test:
    coinbase = None;
    
    def set_coinbase(self, bl=False):
        self.coinbase = bl;
        
    def get_coinbase(self):
        return self.coinbase;
        
        
t = test()
t.set_coinbase(True)
    
t.get_coinbase()

In [ ]:
if not t.get_coinbase():
    print( "False")
else:
    print("true")

In [ ]:
from pyception import AttributeError

In [ ]:
!pip install exceptions